In [1]:
import pandas as pd
import scanpy as sc

In [2]:
# convert to Ensembl protein ID
links = pd.read_csv("../../data/STRING/9606.protein.physical.links.v11.5.txt", sep=" ")
links['protein1'] = links['protein1'].str.split("9606.", expand=True)[1]
links['protein2'] = links['protein2'].str.split("9606.", expand=True)[1]

In [3]:
# convert Ensembl protein ID to Ensembl gene ID
eP2eG = pd.read_csv("../../data/Ensembl/EnsemblP2EnsemblG.txt", sep="\t", header=None).dropna().copy()
eP2eG = eP2eG.set_index(0)[1].to_dict()
links['protein1'] = links['protein1'].map(eP2eG)
links['protein2'] = links['protein2'].map(eP2eG)
links = links.dropna().copy()
links = links[links['protein1'] != links['protein2']].copy() # remove self loop

In [4]:
# valid genes
genes = pd.read_csv("../../result/network_GTEx_phyloP/valid_genes", sep="\t")
id2genes = genes.set_index("ID")["genes"].to_dict()
genes2id = genes.set_index("genes")["ID"].to_dict()
genes = genes['genes'].values

In [5]:
# keep valid genes
links = links[links["protein1"].isin(genes)]
links = links[links["protein2"].isin(genes)]

In [6]:
# convert to number ID
links["protein1"] = links["protein1"].map(genes2id)
links["protein2"] = links["protein2"].map(genes2id)

In [7]:
# uniq and sort
links = links.groupby(["protein1", "protein2"]).mean().reset_index()
links = links.sort_values(['protein1', 'protein2'])

In [8]:
links.to_csv("../../result/network_GTEx_phyloP/STRING_withScore.tsv", sep="\t", header=False, index=False)
links.iloc[:, :2].to_csv("../../result/network_GTEx_phyloP/STRING.tsv", sep="\t", header=False, index=False)